Dependencies

In [ ]:
# Install ifcopenshell for IFC parsing
!pip install ifcopenshell matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 MB 21.8 MB/s eta 0:00:00


Cloning Repo

In [ ]:
!git clone https://github.com/sylvainHellin/ifc-bench.git

Cloning into 'ifc-bench'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 90 (delta 1), reused 6 (delta 0), pack-reused 78 (from 1)
Receiving objects: 100% (90/90), 45.07 MiB | 22.28 MiB/s, done.
Resolving deltas: 100% (10/10), done.
Filtering content: 100% (5/5), 163.22 MiB | 22.39 MiB/s, done.


Tool E Area-Occupancy

In [ ]:
import ifcopenshell
import ifcopenshell.geom
import math

In [ ]:
# Requirements
import math
import json
import ifcopenshell
import ifcopenshell.geom

# --------------------------
# Geometry settings
# --------------------------
settings = ifcopenshell.geom.settings()
settings.set(settings.USE_WORLD_COORDS, True)

# --------------------------
# IFC helpers
# --------------------------
def calculate_space_area(space):
    """Approximate area by summing triangle areas from the space mesh."""
    try:
        shape = ifcopenshell.geom.create_shape(settings, space)
        verts = shape.geometry.verts
        faces = shape.geometry.faces
        area = 0.0

        for i in range(0, len(faces), 3):
            i0, i1, i2 = faces[i] * 3, faces[i + 1] * 3, faces[i + 2] * 3
            v0 = verts[i0:i0 + 3]
            v1 = verts[i1:i1 + 3]
            v2 = verts[i2:i2 + 3]

            a = math.sqrt((v1[0]-v0[0])**2 + (v1[1]-v0[1])**2 + (v1[2]-v0[2])**2)
            b = math.sqrt((v2[0]-v1[0])**2 + (v2[1]-v1[1])**2 + (v2[2]-v1[2])**2)
            c = math.sqrt((v0[0]-v2[0])**2 + (v0[1]-v2[1])**2 + (v0[2]-v2[2])**2)

            s = (a + b + c) / 2.0
            area += math.sqrt(max(s * (s - a) * (s - b) * (s - c), 0.0))

        return float(area)
    except Exception:
        return 0.0


def _space_label(space):
    """Return a readable label."""
    return getattr(space, "LongName", None) or getattr(space, "Name", None) or "Unnamed"


def _matches_any_keyword(text, keywords):
    t = (text or "").lower()
    return any(k in t for k in keywords)


def get_spaces_by_keywords(model, keywords):
    """Filter IfcSpace by semantic keywords."""
    spaces = model.by_type("IfcSpace")
    return [s for s in spaces if _matches_any_keyword(_space_label(s), keywords)]


# --------------------------
# Regulation logic
# --------------------------
def area_to_occupancy(area):
    """
    Mapping regulation:
        <5 m²   -> not valid bedroom
        ≥5 m²   -> 1 person
        ≥8 m²   -> 2 people
        ≥12 m²  -> 3 people
    """
    if area < 5.0:
        return 0
    elif area < 8.0:
        return 1
    elif area < 12.0:
        return 2
    else:
        return 3


def bedroom_occupancy_check(ifc_model_path):
    """
    Determines allowed occupancy from bedroom areas.

    Special rule:
    - If NO bedrooms exist (studio dwelling), occupancy is limited to max 2 people.
    """

    model = ifcopenshell.open(ifc_model_path)

    bedroom_keywords = ["bedroom", "habitacion", "habitación", "dormitorio"]
    living_keywords  = ["living", "salon", "salón", "studio", "estudio", "common"]

    bedrooms = get_spaces_by_keywords(model, bedroom_keywords)
    living_spaces = get_spaces_by_keywords(model, living_keywords)

    room_areas = {}
    occupancy = {}
    total_people = 0

    # --------------------------
    # Studio case (no bedrooms)
    # --------------------------
    if len(bedrooms) == 0:
        if not living_spaces:
            return {
                "result": "fail",
                "reason": "No habitable space identified",
                "room_areas": {},
                "occupancy": {},
                "total_allowed_people": 0
            }

        main_space = living_spaces[0]
        label = _space_label(main_space)
        area = calculate_space_area(main_space)

        allowed = min(area_to_occupancy(area), 2)

        room_areas[label] = float(area)
        occupancy[label] = allowed

        return {
            "result": "pass",
            "reason": "Studio dwelling limited to maximum 2 occupants",
            "room_areas": room_areas,
            "occupancy": occupancy,
            "total_allowed_people": allowed
        }

    # --------------------------
    # Bedroom-based occupancy
    # --------------------------
    for space in bedrooms:
        label = _space_label(space)
        area = calculate_space_area(space)
        allowed = area_to_occupancy(area)

        if allowed == 0:
            return {
                "result": "fail",
                "reason": f"{label} area {area:.2f} m² is below minimum 5 m²",
                "room_areas": room_areas,
                "occupancy": occupancy,
                "total_allowed_people": total_people
            }

        room_areas[label] = float(area)
        occupancy[label] = allowed
        total_people += allowed

    return {
        "result": "pass",
        "reason": "Bedroom areas satisfy occupancy requirements",
        "room_areas": room_areas,
        "occupancy": occupancy,
        "total_allowed_people": total_people
    }


# --------------------------
# Tool entrypoint
# --------------------------
def bedroom_occupancy_check_tool(ifc_model_path: str):
    return bedroom_occupancy_check(ifc_model_path)


# --------------------------
# Schema (LLM-callable description)
# --------------------------
BEDROOM_OCCUPANCY_CHECK_SCHEMA = {
    "name": "bedroom_occupancy_check_tool",
    "description": "Evaluates bedroom areas in an IFC model to determine the legally allowed number of occupants per room and for the dwelling.",
    "parameters": {
        "type": "object",
        "properties": {
            "ifc_model_path": {
                "type": "string",
                "description": "Filesystem path to the IFC model to evaluate."
            }
        },
        "required": ["ifc_model_path"]
    }
}


# --------------------------
# Optional: local sanity test
# --------------------------
if __name__ == "__main__":
    print("Schema OK:")
    print(json.dumps(BEDROOM_OCCUPANCY_CHECK_SCHEMA, indent=2))

    ifc_path = "/content/ifc-bench/projects/duplex/arc.ifc"
    result = bedroom_occupancy_check_tool(ifc_path)

    print(result["result"])
    print(result["reason"])
    print("Allowed occupants:", result["total_allowed_people"])

Schema OK:
{
  "name": "bedroom_occupancy_check_tool",
  "description": "Evaluates bedroom areas in an IFC model to determine the legally allowed number of occupants per room and for the dwelling.",
  "parameters": {
    "type": "object",
    "properties": {
      "ifc_model_path": {
        "type": "string",
        "description": "Filesystem path to the IFC model to evaluate."
      }
    },
    "required": [
      "ifc_model_path"
    ]
  }
}
pass
Bedroom areas satisfy occupancy requirements
Allowed occupants: 12
